In [1]:
# Install required packages.
import os
import torch
os.environ['TORCH'] = torch.__version__
print(torch.__version__)
from tqdm import tqdm, trange

1.11.0+cu113


In [3]:
#!pip install torch-scatter torch-sparse torch-cluster torch-spline-conv torch-geometric -f https://data.pyg.org/whl/torch-1.11.0+cpu.html

In [4]:
import numpy as np
import torch
from torch_geometric.data import HeteroData
import torch_geometric.transforms as T
from torch_geometric.nn import Sequential, Linear
from torch.nn import ReLU
from torch_geometric.datasets import OGB_MAG
from torch_geometric.nn import SAGEConv, to_hetero
from torch_geometric.loader import NeighborLoader, HGTLoader
from torch.nn import Linear
import torch.nn.functional as F
from torch_geometric.nn import GCNConv #GATConv

In [5]:
dataset = OGB_MAG(root='./data', preprocess='metapath2vec', transform=T.ToUndirected())
data = dataset[0]

Extracting data/mag/raw/mag.zip
Extracting data/mag/raw/mag_metapath2vec_emb.zip
Processing...
Done!


In [6]:
class GNN(torch.nn.Module):
    def __init__(self, hidden_channels, out_channels):
        super().__init__()
        self.conv1 = SAGEConv((-1, -1), hidden_channels)
        self.conv2 = SAGEConv((-1, -1), out_channels)

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index).relu()
        x = self.conv2(x, edge_index)
        return x


model = GNN(hidden_channels=64, out_channels=dataset.num_classes)
model = to_hetero(model, data.metadata(), aggr='sum')

In [7]:
data.metadata()

(['paper', 'author', 'institution', 'field_of_study'],
 [('author', 'affiliated_with', 'institution'),
  ('author', 'writes', 'paper'),
  ('paper', 'cites', 'paper'),
  ('paper', 'has_topic', 'field_of_study'),
  ('institution', 'rev_affiliated_with', 'author'),
  ('paper', 'rev_writes', 'author'),
  ('field_of_study', 'rev_has_topic', 'paper')])

In [15]:
data

HeteroData(
  paper={
    x=[736389, 128],
    year=[736389],
    y=[736389],
    train_mask=[736389],
    val_mask=[736389],
    test_mask=[736389]
  },
  author={ x=[1134649, 128] },
  institution={ x=[8740, 128] },
  field_of_study={ x=[59965, 128] },
  (author, affiliated_with, institution)={ edge_index=[2, 1043998] },
  (author, writes, paper)={ edge_index=[2, 7145660] },
  (paper, cites, paper)={ edge_index=[2, 10792672] },
  (paper, has_topic, field_of_study)={ edge_index=[2, 7505078] },
  (institution, rev_affiliated_with, author)={ edge_index=[2, 1043998] },
  (paper, rev_writes, author)={ edge_index=[2, 7145660] },
  (field_of_study, rev_has_topic, paper)={ edge_index=[2, 7505078] }
)

In [8]:
model

GraphModule(
  (conv1): ModuleDict(
    (author__affiliated_with__institution): SAGEConv((-1, -1), 64)
    (author__writes__paper): SAGEConv((-1, -1), 64)
    (paper__cites__paper): SAGEConv((-1, -1), 64)
    (paper__has_topic__field_of_study): SAGEConv((-1, -1), 64)
    (institution__rev_affiliated_with__author): SAGEConv((-1, -1), 64)
    (paper__rev_writes__author): SAGEConv((-1, -1), 64)
    (field_of_study__rev_has_topic__paper): SAGEConv((-1, -1), 64)
  )
  (conv2): ModuleDict(
    (author__affiliated_with__institution): SAGEConv((-1, -1), 349)
    (author__writes__paper): SAGEConv((-1, -1), 349)
    (paper__cites__paper): SAGEConv((-1, -1), 349)
    (paper__has_topic__field_of_study): SAGEConv((-1, -1), 349)
    (institution__rev_affiliated_with__author): SAGEConv((-1, -1), 349)
    (paper__rev_writes__author): SAGEConv((-1, -1), 349)
    (field_of_study__rev_has_topic__paper): SAGEConv((-1, -1), 349)
  )
)

In [11]:
# Use GPU
#device = torch.device("cpu")
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

model = model.to(device)
data = data.to(device)

# Initialize Optimizer
learning_rate = 0.01
decay = 5e-4
optimizer = torch.optim.Adam(model.parameters(), 
                             lr=learning_rate, 
                             weight_decay=decay)
# Define loss function (CrossEntropyLoss for Classification Problems with 
# probability distributions)
#criterion = torch.nn.CrossEntropyLoss()

In [12]:
def train():
      model.train()
      optimizer.zero_grad() 
      # Use all data as input, because all nodes have node features
      out = model(data.x_dict, data.edge_index_dict)  
      # Only use nodes with labels available for loss calculation --> mask
      mask = data['paper'].train_mask
      loss = F.cross_entropy(out['paper'][mask], data['paper'].y[mask])
      loss.backward() 
      optimizer.step()
      return loss

def test():
      model.eval()
      out = model(data.x, data.edge_index)
      # Use the class with highest probability.
      pred = out.argmax(dim=1)  
      # Check against ground-truth labels.
      test_correct = pred[data.test_mask] == data.y[data.test_mask]  
      # Derive ratio of correct predictions.
      test_acc = int(test_correct.sum()) / int(data.test_mask.sum())  
      return test_acc

In [14]:
losses = []
for epoch in range(0, 20):
    loss = train()
    losses.append(loss)
    print(f'Epoch: {epoch:03d}, Loss: {loss:.4f}')

Epoch: 000, Loss: 3.3653
Epoch: 001, Loss: 3.2315
Epoch: 002, Loss: 3.1098
Epoch: 003, Loss: 3.0121
Epoch: 004, Loss: 2.9300
Epoch: 005, Loss: 2.8586
Epoch: 006, Loss: 2.8009
Epoch: 007, Loss: 2.7534
Epoch: 008, Loss: 2.7079
Epoch: 009, Loss: 2.6704
Epoch: 010, Loss: 2.6368
Epoch: 011, Loss: 2.6024
Epoch: 012, Loss: 2.5728
Epoch: 013, Loss: 2.5452
Epoch: 014, Loss: 2.5205
Epoch: 015, Loss: 2.4997
Epoch: 016, Loss: 2.4820
Epoch: 017, Loss: 2.4640
Epoch: 018, Loss: 2.4457
Epoch: 019, Loss: 2.4255


In [ ]:
device

device(type='cuda', index=0)

In [ ]:
data.x_dict

{'author': tensor([[-0.4683,  0.1084, -0.0180,  ..., -0.2873,  0.3973,  0.0373],
         [ 0.1035, -0.3703, -0.3722,  ...,  0.5777,  0.0044, -0.3645],
         [ 0.3745,  0.0797,  0.3995,  ...,  0.0166, -0.5806, -0.1265],
         ...,
         [-0.0076,  0.6291,  0.0684,  ...,  0.0279,  0.1603, -0.0225],
         [ 0.1657, -0.1814,  0.2352,  ..., -0.4000, -0.4608, -0.7904],
         [-0.4098,  0.0470, -0.2027,  ...,  0.1393, -0.1985, -0.6175]],
        device='cuda:0'),
 'field_of_study': tensor([[ 1.7532e-02, -3.0144e-01,  3.2530e-01,  ..., -1.6283e-01,
          -1.3862e-01,  3.3216e-01],
         [-5.3028e-01,  1.5095e-01, -1.2914e-01,  ...,  7.6167e-02,
           3.0035e-01, -1.5220e-02],
         [-7.1854e-01,  6.7117e-04, -4.1465e-01,  ..., -1.5486e-01,
          -4.1101e-01, -6.1223e-01],
         ...,
         [-3.1354e-01,  8.0473e-01,  9.4273e-02,  ..., -3.2736e-01,
           7.9512e-01, -5.7828e-02],
         [-5.5001e-01,  1.7064e-01, -3.7380e-01,  ..., -2.2461e-01,
   

In [ ]:
data.edge_index_dict

{('author',
  'affiliated_with',
  'institution'): tensor([[      0,       1,       2,  ..., 1134645, 1134647, 1134648],
         [    845,     996,    3197,  ...,    5189,    4668,    4668]],
        device='cuda:0'),
 ('author',
  'writes',
  'paper'): tensor([[      0,       0,       0,  ..., 1134647, 1134648, 1134648],
         [  19703,  289285,  311768,  ...,  657395,  671118,  719594]],
        device='cuda:0'),
 ('field_of_study',
  'rev_has_topic',
  'paper'): tensor([[   145,   2215,   3205,  ...,  21458,  22283,  31934],
         [     0,      0,      0,  ..., 736388, 736388, 736388]],
        device='cuda:0'),
 ('institution',
  'rev_affiliated_with',
  'author'): tensor([[    845,     996,    3197,  ...,    5189,    4668,    4668],
         [      0,       1,       2,  ..., 1134645, 1134647, 1134648]],
        device='cuda:0'),
 ('paper',
  'cites',
  'paper'): tensor([[     0,      0,      0,  ..., 736388, 736388, 736388],
         [    88,  27449, 121051,  ..., 707740, 7